In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ln -s "/content/drive/MyDrive/Group_11/" link

Mounted at /content/drive


In [ ]:
#Extracting train images packet

!mkdir -p /content/train

%cd '/content/train'
!unrar x "/content/link/trainset/train2401_2800.rar"

Output streaming troncato alle ultime 5000 righe.
Extracting  n002784/0103_01.jpg                                           96%  OK 
Extracting  n002784/0104_02.jpg                                           96%  OK 
Extracting  n002784/0105_01.jpg                                           96%  OK 
Extracting  n002784/0106_01.jpg                                           96%  OK 
Extracting  n002784/0107_01.jpg                                           96%  OK 
Extracting  n002784/0108_01.jpg                                           96%  OK 
Extracting  n002784/0109_01.jpg                                           96%  OK 
Extracting  n002784/0110_01.jpg                                           96%  OK 
Extracting  n002784/0111_01.jpg                                           96%  OK 
Extracting  n002784/0112_01.jpg                                           96%  OK 
Extracting  n002784/0113_01.jp

In [ ]:
# Reading dataframe from csv
import pandas as pd
import os

# Import the csv with filepaths and labels
traindf=pd.read_csv('/content/link/trainset/train2401_2800.csv', dtype='string', index_col=0)

# Round to integers, filter 50000 samples
traindf = traindf.astype({'age': float})
traindf['age'] = traindf['age'].apply(lambda x: int(round(x, 0)))
traindf = traindf.astype({'age':'string'})
traindf = traindf[:50000]
traindf = traindf.reset_index(drop=True)

# Build the vector of classes
classes = [str(i) for i in range(0, 101)]

# Prepare the training/validation split
TRAIN_SPLIT = 0.95
VALID_SPLIT = 1 - TRAIN_SPLIT


print(traindf)
print(traindf.dtypes)
print("Train:", int(TRAIN_SPLIT*traindf.shape[0]), "samples; Validation:", int(VALID_SPLIT*traindf.shape[0]), "samples.")

      age             filepath
0      31  n002401/0001_01.jpg
1      31  n002401/0002_01.jpg
2      30  n002401/0003_01.jpg
3      28  n002401/0004_01.jpg
4      30  n002401/0005_01.jpg
...    ..                  ...
49995  28  n002547/0195_01.jpg
49996  25  n002547/0196_01.jpg
49997  26  n002547/0197_01.jpg
49998  28  n002547/0199_01.jpg
49999  29  n002547/0200_01.jpg

[50000 rows x 2 columns]
age         string
filepath    string
dtype: object
Train: 47500 samples; Validation: 2500 samples.


In [ ]:
# Install the detector
!pip install mtcnn

# Install the model lib
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_vggface
!pip install keras_applications

     |████████████████████████████████| 2.3MB 7.2MB/s 
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-zd2_3sys
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-zd2_3sys
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8310 sha256=906dbbfc8b513551ef7d9f20f75f78dda22616c1e8be9001de4c7e34037402d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-khelc7j2/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
     |████████████████████████████████| 51kB 2.5MB/s 


In [ ]:
#Steps for image preprocessing

from imutils.face_utils import FaceAligner
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import imutils
import dlib
from imutils.face_utils import rect_to_bb
from keras_vggface import utils
from mtcnn import MTCNN

# initialize mtcnn face detector, then create
# the facial landmark predictor and the face aligner
detector = MTCNN()
predictor = dlib.shape_predictor('/content/link/shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=224)


# Preprocessing function to crop face and align it
def preprocess_image(image, plot=False):
  # load the input image and convert it to grayscale
  image = image.astype(np.uint8)
  gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)

  # find all the faces in the image
  rects = []
  for faces in detector.detect_faces(image):
    box = faces["box"]
    rects.append(dlib.rectangle(box[0], box[1], box[0]+box[2], box[1]+box[3]))
  max = 0

  # loop over the face detections to find the largest one
  for rect in rects:
    (x, y, w, h) = rect_to_bb(rect)
    area = w*h
    if area > max:
      max = area
      (x_m, y_m, w_m, h_m) = (x, y, w, h)
      rect_m = rect

  # if at least one face was found, align it
  if max!=0:
    faceAligned = fa.align(image, gray, rect_m)
    if plot:
      plt.imshow(faceAligned)
      plt.show()
    return faceAligned
  
  # no face revealed
  return None

In [ ]:
# GENERATOR 1: Defining the IMAGE GENERATOR for the training
# this generator works creating in sequence one batch of 
# original images and one batch of augmented images
# 
# **IMPORTANT: This generator does not increment the number of samples**
from keras.preprocessing.image import ImageDataGenerator
from random import randint
import math

class Generator():
  def __init__(self, dataframe, batch_size, classes, val_split=0.2):
    self.val_split = val_split
    self.classes = classes
    self.flag = True
    self.dataframe = dataframe
    self.dim_val = int(dataframe.shape[0]*val_split)
    self.gap = math.floor(batch_size/4)

    # create a main generator for the training and validation
    self.gen = ImageDataGenerator(validation_split=val_split)
    self.it_train = self.gen.flow_from_dataframe(dataframe = dataframe, directory='/content/train', x_col="filepath", y_col="age", classes=self.classes, target_size=(224, 224), color_mode="rgb", shuffle = True, batch_size = batch_size, subset='training')
    self.it_val = self.gen.flow_from_dataframe(dataframe = dataframe, directory='/content/train', x_col="filepath", y_col="age", classes=self.classes, target_size=(224, 224), color_mode="rgb", shuffle = False, batch_size = self.dim_val, subset='validation')
    
    # define 4 generators to create 4 different augmented images
    self.gen1 = ImageDataGenerator(preprocessing_function = self.preprocess_keras, height_shift_range = [-50, 50], brightness_range = [0.8, 1.2])
    self.gen2 = ImageDataGenerator(preprocessing_function = self.preprocess_keras, brightness_range = [0.2, 1.8], width_shift_range = [-50, 50])
    self.gen3 = ImageDataGenerator(preprocessing_function = self.random_blur)
    self.gen4 = ImageDataGenerator(preprocessing_function = self.gaussian_noise)

    # emergence generator in case no face found in the image
    self.gen_em = ImageDataGenerator()


  def __iter__(self):
    return self
  
  def __next__(self):
    # take a batch of original images
    images, labels = next(self.it_train)
    g=self.gap

    # crop faces and align, then call preprocess_keras function
    for i in range(images.shape[0]):
      while(True):
        image = preprocess_image(images[i])
        if image is not None:
          images[i] = image
          break
        images[i], labels[i] = next(self.gen_em.flow_from_dataframe(dataframe = self.dataframe, directory='/content/train', x_col="filepath", y_col="age", classes = self.classes, target_size=(224, 224), color_mode="rgb", batch_size = 1, shuffle = False, subset='training'))
      images[i] = self.preprocess_keras(images[i])
    

    # first batch: original images
    if self.flag:
      self.flag = False
      return (images, labels)

    # next batch: augmented images
    else:
      self.flag = True
      images[0:g], labels[0:g] = next(self.gen1.flow(images[0:g], labels[0:g], batch_size = g, shuffle = False))
      images[g:2*g], labels[g:2*g] = next(self.gen2.flow(images[g:2*g], labels[g:2*g], batch_size = g, shuffle = False))
      images[2*g:3*g], labels[2*g:3*g] = next(self.gen3.flow(images[2*g:3*g], labels[2*g:3*g], batch_size = g, shuffle = False))
      images[3*g:], labels[3*g:] = next(self.gen4.flow(images[3*g:], labels[3*g:], batch_size = g, shuffle = False))
      return (images, labels)


  # function to obtain a batch of validating images
  def next_val(self):
    print("Preparing validation images...")
    val_images = np.empty((0,224,224,3))
    val_labels = np.empty((0, len(self.classes)))
    images, labels = next(self.it_val)

    # preprocess the images
    for i in range(images.shape[0]):
      image = preprocess_image(images[i])
      if image is None:
        continue
      
      image = self.preprocess_keras(image)
      image = image[np.newaxis, :]
      label = labels[i]
      label = label[np.newaxis, :]
      val_images = np.append(val_images, image, axis=0)
      val_labels = np.append(val_labels, label, axis=0)

      # print status
      en_ticks = int(30*(i+1)/images.shape[0])
      space_ticks = 30 - en_ticks
      print("\rLOADING: |"+('='*en_ticks)+(' '*space_ticks)+'|', end='')

    print(" Done")
    return (val_images, val_labels)


  # function to convert face images into input for VGG-Face
  def preprocess_keras(self, image):
    return utils.preprocess_input(image.astype('float64'), version=2)

  # function to apply a random blur on the face image
  def random_blur(self, image):
    value1 = randint(2, 8)
    value2 = randint(2, 8)
    image = cv.blur(image, (value1, value2))
    return utils.preprocess_input(image.astype('float64'), version=2)

  # function to apply a random gaussian noise on the face image
  def gaussian_noise(self, image):
    row,col,ch= image.shape
    mean = 0
    var = 50
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return utils.preprocess_input(noisy.astype('float64'), version=2)
    

In [ ]:
# GENERATOR 2: Defining the IMAGE GENERATOR for Data Augmentation
# 
# This generator increments the number of samples: for each image,
# it creates other 5 images
from keras.preprocessing.image import ImageDataGenerator
import random

class Generator_AUG():
  def __init__(self, dataframe, batch_size, classes, val_split=0.2):
    self.val_split = val_split
    self.classes = classes
    self.flag = True
    self.dataframe = dataframe
    self.dim_val = int(dataframe.shape[0]*val_split)
    self.gap = int(batch_size/6)

    # create the main generator
    self.gen = ImageDataGenerator(validation_split=val_split)
    self.it_train = self.gen.flow_from_dataframe(dataframe = dataframe, directory='/content/train', x_col="filepath", y_col="age", classes=self.classes, target_size=(224, 224), color_mode="rgb", shuffle = True, batch_size = self.gap, subset='training')
    self.it_val = self.gen.flow_from_dataframe(dataframe = dataframe, directory='/content/train', x_col="filepath", y_col="age", classes=self.classes, target_size=(224, 224), color_mode="rgb", shuffle = False, batch_size = self.dim_val, subset='validation')
    
    # 5 generators for augmented images
    self.gen1 = ImageDataGenerator(preprocessing_function = self.preprocess_keras, height_shift_range = [-50, 50], brightness_range = [0.8, 1.2])
    self.gen2 = ImageDataGenerator(preprocessing_function = self.preprocess_keras, brightness_range = [0.2, 1.8], width_shift_range = [-50, 50])
    self.gen3 = ImageDataGenerator(preprocessing_function = self.random_blur)
    self.gen4 = ImageDataGenerator(preprocessing_function = self.gaussian_noise)
    self.gen5 = ImageDataGenerator(preprocessing_function = self.sp_noise)
    self.gen_em = ImageDataGenerator()


  def __iter__(self):
    return self
  
  # function to return a batch of images
  def __next__(self):
    # take a mini-batch
    images, labels = next(self.it_train)

    # preprocess images
    for i in range(images.shape[0]):
      while(True):
        image = preprocess_image(images[i])
        if image is not None:
          images[i] = image
          break
        images[i], labels[i] = next(self.gen_em.flow_from_dataframe(dataframe = self.dataframe, directory='/content/train', x_col="filepath", y_col="age", classes = self.classes, target_size=(224, 224), color_mode="rgb", batch_size = 1, shuffle = False, subset='training'))
      images[i] = self.preprocess_keras(images[i])
    
    g = self.gap
    # modify images, creating other 5 images
    im, lab = next(self.gen1.flow(images, labels, batch_size = g, shuffle = False))
    images = np.append(images, im, axis=0)
    labels = np.append(labels, lab, axis=0) 
    im, lab = next(self.gen2.flow(images, labels, batch_size = g, shuffle = False))
    images = np.append(images, im, axis=0)
    labels = np.append(labels, lab, axis=0) 
    im, lab = next(self.gen3.flow(images, labels, batch_size = g, shuffle = False))
    images = np.append(images, im, axis=0)
    labels = np.append(labels, lab, axis=0)
    im, lab = next(self.gen4.flow(images, labels, batch_size = g, shuffle = False))
    images = np.append(images, im, axis=0)
    labels = np.append(labels, lab, axis=0)
    im, lab = next(self.gen5.flow(images, labels, batch_size = g, shuffle = False))
    images = np.append(images, im, axis=0)
    labels = np.append(labels, lab, axis=0)

    return (images, labels)

  # function to obtain a batch of validating images
  def next_val(self):
    print("Preparing validation images...")
    val_images = np.empty((0,224,224,3))
    val_labels = np.empty((0, len(self.classes)))
    images, labels = next(self.it_val)

    # preprocess images
    for i in range(images.shape[0]):
      image = preprocess_image(images[i])
      if image is None:
        continue
      
      image = self.preprocess_keras(image)
      image = image[np.newaxis, :]
      label = labels[i]
      label = label[np.newaxis, :]
      val_images = np.append(val_images, image, axis=0)
      val_labels = np.append(val_labels, label, axis=0)

      en_ticks = int(30*(i+1)/images.shape[0])
      space_ticks = 30 - en_ticks
      print("\rLOADING: |"+('='*en_ticks)+(' '*space_ticks)+'|', end='')


    print(" Done")
    return (val_images, val_labels)

  # function to convert face images into input for VGG-Face
  def preprocess_keras(self, image):
    return utils.preprocess_input(image.astype('float64'), version=2)

  # function to apply a random blur on the face image
  def random_blur(self, image):
    value1 = random.randint(2, 8)
    value2 = random.randint(2, 8)
    image = cv.blur(image, (value1, value2))
    return utils.preprocess_input(image.astype('float64'), version=2)

  # function to apply a random gaussian noise on the face image
  def gaussian_noise(self, image):
    row,col,ch= image.shape
    mean = 0
    var = 50
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return utils.preprocess_input(noisy.astype('float64'), version=2)

  # function to apply a random sp noise on the face image
  def sp_noise(self, image):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    prob = 0.01
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return utils.preprocess_input(output.astype('float64'), version=2)
    

NOW WE START THE BUILDING OF THE MODEL AND ITS TRAINING

In [ ]:
# run this block to build the initial model or skip to the next code block
# TRANSFER LEARNING has been used, so we added 2 additional layers

from keras.engine import Model
from keras_vggface.vggface import VGGFace
from keras.layers import Flatten, Dense
vggface = VGGFace(model='resnet50', input_shape=(224, 224, 3))

# output dim = numb of classes
output_dim = 101

# freeze all the layers of VGG-Face
for layer in vggface.layers:
  layer.trainable = False

# add layers and compile the model
output_layer = vggface.get_layer('avg_pool').output
x = Flatten(name="flatten")(output_layer)
x = Dense(1024, name="fc7", activation="relu")(x)
out = Dense(output_dim, name="fc8", activation="softmax")(x)
model = Model(vggface.input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_crossentropy"])

#print(model.summary())

165445632/165439116 [==============================] - 2s 0us/step


In [ ]:
# run this block to import a presaved model
from tensorflow import keras

model = keras.models.load_model('/content/link/model.h5')

In [ ]:
# Training the model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import cv2 as cv
import numpy as np
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%d:%m:%Y-%H:%M:%S")

# Choose parameters
BATCH_SIZE = 60   # set a number divisible for 4
EPOCHS = 5
STEPS_PER_EPOCH = traindf.shape[0]*TRAIN_SPLIT//(BATCH_SIZE*EPOCHS)

# path where to save the trained model
checkpoint_filepath = "/content/link/saved_models/model." + current_time + ".{epoch:02d}-{loss:.3f}.h5"

model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, monitor="val_loss")
reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)

callbacks = [model_checkpoint, reduce_on_plateau]

# create the generator and fit the model
train_generator = Generator(traindf, BATCH_SIZE, classes, val_split=VALID_SPLIT)
model.fit(train_generator, validation_data=train_generator.next_val(), callbacks=callbacks, batch_size=BATCH_SIZE, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH)

NOW, THE MODEL IS FINISHED, SO WE CAN TEST WITH SOME DATA

In [ ]:
import cv2 as cv

# DEFINING FUNCTION TO PREDICT AGE FROM IMAGE USING OUR MODEL

'''
Returns age and its probability
img: image
plot: if True, plots the face
'''
def model_predict(img, plot=False):
  img=preprocess_image(img, plot)
  img=utils.preprocess_input(img.astype('float64'),version=2) #ritorna il file come lo vuole la rete 
  img = img[np.newaxis, :]
  classes = model.predict(img)
  index = np.argmax(classes)
  return (index, classes[0][index])

'''
Returns the mae on a batch of images
X: images list
Y: labels list
'''
def model_accuracy(X, Y):

  if X.shape[0] == 0:
    return 0

  error = 0
  for i in range(X):
    error += abs(int(Y[i]) - int(model_predict(Y[i])[0])) 
  return error/X.shape[0]

'''
Return the mae on a dataframe
df: dataframe
plot: if True, plots the face
'''
def model_accuracy_from_dataframe(df, plot=False):

  if df.shape[0] == 0:
    return 0

  error = 0
  i = 0
  for index, row in df.iterrows():
    path = row['filepath']
    label = row['age']

    img=cv.imread(path)
    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    est = model_predict(img, plot)[0]
    print("true:" + label + " Estimated:" + str(est))
    error += abs(int(label) - est)

  return error/df.shape[0]


In [ ]:
# PREPARE A TEST DATAFRAME AND PRINT ACCURACY
#print(model_accuracy_from_dataframe(testdf, True))